In [1]:
import numpy as np
import os

In [2]:
buffer_size = 10000000

In [3]:
def save_array_to_file(file_name, array_to_save):
    np.savetxt(file_name, array_to_save, fmt = '%d')
    
def sort_and_write(file_name, array_to_sort):
    array_to_sort.sort()
    save_array_to_file(file_name, array_to_sort)

In [ ]:
def read_x_int(file_, numbers_to_read):
    array_ = []
    num = file_.readline()
    while(num and len(array_) < numbers_to_read):
        if len(next_str) > 0:
            
        num = file_.readline()
    return array_

In [11]:
arr = np.arange(300000000)
np.random.shuffle(arr)

In [12]:
%%time
save_array_to_file('unsorted.csv', arr)
arr = None

Wall time: 13min 49s


In [13]:
%%time
read_arr = []

count = 0
chunk = 1
with open('unsorted.csv') as f:
    num = f.readline()
    while (num):
        read_arr.append(int(num))
        
        count = count + 1
        if count % buffer_size == 0:
            sort_and_write('./tmp/sorted_' + str(chunk), read_arr)
            read_arr = []
            chunk = chunk + 1
        num = f.readline()
if len(read_arr) > 0:
    sort_and_write('./tmp/sorted_' + str(chunk), read_arr)

Wall time: 20min 54s


In [14]:
%%time
import heapq

sorted_file = open('sorted_external.csv', 'w+')

files = [f for f in os.listdir('./tmp/') if os.path.isfile('./tmp/' + f)]

list_opened_files = [open('./tmp/' + file_) for file_ in files]

min_heap = []
heapq.heapify(min_heap)

for file_ in list_opened_files:
    heapq.heappush(min_heap, (int(file_.readline()), file_))

while(len(min_heap) > 0):
    min_element = heapq.heappop(min_heap)
    sorted_file.write(str(min_element[0]) + '\n')
    next_str = min_element[1].readline()
    if next_str:
        heapq.heappush(min_heap, (int(next_str), min_element[1]))
    
sorted_file.close()

[f.close() for f in list_opened_files]

Wall time: 15min 15s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [15]:
read_arr = None

In [4]:
%%time
read_arr = []

with open('unsorted.csv') as f:
    num = f.readline()
    while (num):
        read_arr.append(int(num))
        num = f.readline()

Wall time: 3min 10s


In [ ]:
%%time
read_arr.sort()

In [ ]:
%%time
save_array_to_file('sorted_in_mem.csv', read_arr)